## How to run the code, and demo shows from raw input string to predictions

In [3]:
from __future__ import print_function
import pandas as pd
import torch

from sectionize_docs import *
from src.n_proc import *

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# specifiy device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# read in the sample csv file
df = pd.read_csv('Anna.csv')

In [ ]:
df['text'] = df['Full Text']
# extract the sections
ih, ap = get_sections(list(df['text']))
df['ih'] = ih
df['ap'] = ap

sec = delete_sections(list(df['text']))
df['sec_text'] = sec

In [6]:
# preprocess text
df['text'] = df['ih'].str.cat(df['ap'], sep=' ')
df['text'] = df.apply(text2sec, axis=1)
df['text'] = df.apply(text_preproc, axis=1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# note to Anna: you need to change this to your own path
PATH = ''
model_path = PATH
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [8]:
# doing prediction one by one here because I am not sure your inputs therefore i do not know how to batch it 
# x will be a string of text here
result = []
for x in df.text:
    input_ids = tokenizer.encode(x, truncation=True, padding=True, return_tensors="pt")
    # Make prediction
    outputs = model(input_ids)
    probas = torch.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probas, dim=1).item()
    # Print predicted class
    # print("Predicted class:", predicted_class)
    result.append(predicted_class)

In [9]:
# the correct result here should be [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0]
print(result)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0]
